In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Load the dataset
prompts_df = pd.read_csv('../data/ShareGPT/unfiltered_english_prompts.csv', header=None)

# Display the first few rows to understand the structure
print(f"Dataset shape: {prompts_df.shape}")
print("\nFirst few rows:")
prompts_df.head()


Dataset shape: (198542, 3)

First few rows:


,0,1,2
0,conversation_id,prompt,prompt_length
1,0,How to tell if a customer segment is well segm...,72
2,1,"In Java, I want to replace string like ""This i...",191
3,2,how do I add multiple new columns in m for pow...,67
4,2,can you show me the m code?,27


In [5]:
# Load the testing prompts dataset
testing_prompts_df = pd.read_csv('../data/ShareGPT/testing_prompts_cleaned.csv')

# Display the first few rows of testing prompts
print(f"Testing prompts shape: {testing_prompts_df.shape}")
print("\nFirst few rows of testing prompts:")
testing_prompts_df.head()

# Extract the original prompts from testing dataset
original_prompts = testing_prompts_df['original_prompt'].tolist()

# Drop rows from prompts_df that match any original prompt in testing_prompts_df
# First, rename columns for easier reference
prompts_df.columns = ['conversation_id', 'prompt', 'prompt_length']

# Filter out rows where the prompt matches any in the testing dataset
prompts_df = prompts_df[~prompts_df['prompt'].isin(original_prompts)]

print(f"\nDataset shape after removing testing prompts: {prompts_df.shape}")



Testing prompts shape: (1000, 4)

First few rows of testing prompts:

Dataset shape after removing testing prompts: (196885, 3)


In [6]:
# Group by conversation_id and take the first prompt for each conversation
first_prompts_df = prompts_df.groupby('conversation_id').first().reset_index()

print(f"Dataset shape after keeping only first prompts: {first_prompts_df.shape}")
print("\nFirst few rows of first prompts:")
first_prompts_df.head()


Dataset shape after keeping only first prompts: (34359, 3)

First few rows of first prompts:


,conversation_id,prompt,prompt_length
0,0,How to tell if a customer segment is well segm...,72
1,1,"In Java, I want to replace string like ""This i...",191
2,10,What is a formal but simplified explanation of...,60
3,100,give me an mountain vector illustration,39
4,1000,Bro I need you to adopt the persona of a super...,58


In [12]:
# Save the first prompts to a CSV file for training
output_path = Path('../data/Prompt_Training_2.0/Candidate_train_data.csv')

# Create directory if it doesn't exist
output_path.parent.mkdir(parents=True, exist_ok=True)

# Save to CSV without the index
first_prompts_df.to_csv(output_path, index=False)

print(f"Saved {len(first_prompts_df)} candidate training prompts to {output_path}")


Saved 34358 candidate training prompts to ..\data\Prompt_Training_2.0\Candidate_train_data.csv


In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import os
import random
import datetime

class PromptSeparator:
    def __init__(self):
        self.prompts_data = []
        self.current_prompt = None
        self.current_index = 0
        self.source_data = []
        self.total_to_process = 10  # Default number of examples to process
        self.processed_ids = set()  # Track IDs we've already processed
        
        # Source file path
        self.source_file_path = os.path.join("..", "data", "prompt_training_2.0", "candidate_train_data.csv")
        
        # Load source data
        self.load_source_data()
        
        # Create widgets
        self.num_examples_input = widgets.IntText(
            value=10,
            description='# Examples:',
            disabled=False,
            layout=widgets.Layout(width='150px')
        )
        
        self.filename_input = widgets.Text(
            value='',
            placeholder='separated_prompts.csv (leave empty for timestamp)',
            description='Save As:',
            disabled=False,
            layout=widgets.Layout(width='400px')
        )
        
        self.load_button = widgets.Button(
            description='Load Examples',
            button_style='primary',
            tooltip='Load random examples from source data',
            disabled=False
        )
        
        self.load_file_button = widgets.Button(
            description='Load Existing File',
            button_style='primary',
            tooltip='Load existing file to append to',
            disabled=False
        )
        
        self.prompt_text = widgets.Textarea(
            value='',
            placeholder='Prompt will appear here',
            description='Prompt:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='200px')
        )
        
        self.context_text = widgets.Textarea(
            value='',
            placeholder='Copy and paste the context portion here',
            description='Context:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )
        
        self.instruction_preview = widgets.Textarea(
            value='',
            placeholder='The instruction part will appear here after extracting context',
            description='Instruction:',
            disabled=True,
            layout=widgets.Layout(width='100%', height='100px')
        )
        
        self.conversation_id_text = widgets.Text(
            value='',
            description='Conversation ID:',
            disabled=True,
            layout=widgets.Layout(width='100%')
        )
        
        self.extract_button = widgets.Button(
            description='Extract Context',
            button_style='info',
            tooltip='Extract the pasted context from the prompt',
            disabled=False
        )
        
        self.no_context_button = widgets.Button(
            description='No Context',
            button_style='warning',
            tooltip='Mark this prompt as having no context',
            disabled=False
        )
        
        self.skip_button = widgets.Button(
            description='Skip',
            button_style='danger',
            tooltip='Skip this example and load next',
            disabled=False
        )
        
        self.next_button = widgets.Button(
            description='Save & Next',
            button_style='success',
            tooltip='Save current prompt and move to next',
            disabled=False
        )
        
        self.export_button = widgets.Button(
            description='Export to CSV',
            button_style='danger',
            tooltip='Export all processed prompts to CSV',
            disabled=False
        )
        
        self.progress = widgets.IntProgress(
            value=0,
            min=0,
            max=10,
            description='Progress:',
            bar_style='info',
            orientation='horizontal'
        )
        
        self.status_text = widgets.HTML(
            value="<p>Ready to load examples</p>"
        )
        
        self.preview = widgets.HTML(
            value="<p><b>Preview:</b></p><p>No prompt processed yet</p>"
        )
        
        # Connect button callbacks
        self.num_examples_input.observe(self.update_progress_max, names='value')
        self.load_button.on_click(self.load_examples)
        self.load_file_button.on_click(self.load_existing_file)
        self.extract_button.on_click(self.extract_context)
        self.no_context_button.on_click(self.mark_no_context)
        self.skip_button.on_click(self.skip_prompt)
        self.next_button.on_click(self.next_prompt)
        self.export_button.on_click(self.export_to_csv)
        
        # Layout
        self.app = widgets.VBox([
            widgets.HTML("<h2>Prompt Context Separator</h2>"),
            widgets.HTML("<p>Instructions: Load examples, copy/paste the context portion, then extract context.</p>"),
            widgets.HBox([
                widgets.VBox([
                    self.num_examples_input,
                    self.filename_input,
                ]),
                widgets.VBox([
                    self.load_button,
                    self.load_file_button
                ])
            ]),
            self.progress,
            self.conversation_id_text,
            self.prompt_text,
            widgets.HBox([
                widgets.VBox([
                    self.context_text,
                    widgets.HBox([self.extract_button, self.no_context_button])
                ]),
                widgets.VBox([
                    self.instruction_preview
                ])
            ]),
            widgets.HBox([
                self.skip_button,
                self.next_button, 
                self.export_button
            ]),
            self.status_text,
            widgets.HTML("<h3>Processing Preview</h3>"),
            self.preview
        ])
    
    def load_source_data(self):
        """Load data from the specific source file"""
        try:
            if not os.path.exists(self.source_file_path):
                print(f"Source file not found: {self.source_file_path}")
                return
            
            # Load the CSV file
            df = pd.read_csv(self.source_file_path)
            self.source_data = df.to_dict('records')
            
            print(f"Loaded {len(self.source_data)} prompts from {self.source_file_path}")
        except Exception as e:
            print(f"Error loading source data: {str(e)}")
    
    def load_existing_file(self, b):
        """Load an existing file to append to"""
        filename = self.filename_input.value.strip()
        if not filename:
            self.status_text.value = "<p style='color:red;'>Please enter a filename first.</p>"
            return
            
        if not filename.endswith('.csv'):
            filename += '.csv'
            
        filepath = os.path.join("..", "data", "prompt_training_2.0", filename)
        
        if not os.path.exists(filepath):
            self.status_text.value = f"<p style='color:orange;'>File {filename} doesn't exist yet. It will be created when you export.</p>"
            return
            
        try:
            df = pd.read_csv(filepath)
            self.prompts_data = df.to_dict('records')
            
            # Extract processed IDs to avoid duplicates
            self.processed_ids = set()
            for item in self.prompts_data:
                if 'conversation_id' in item:
                    self.processed_ids.add(str(item['conversation_id']))
                    
            self.status_text.value = f"<p style='color:green;'>Loaded {len(self.prompts_data)} existing prompts from {filename}. Will append new prompts.</p>"
        except Exception as e:
            self.status_text.value = f"<p style='color:red;'>Error loading file: {str(e)}</p>"
    
    def update_progress_max(self, change):
        """Update the progress bar max when number of examples changes"""
        self.total_to_process = change.new
        self.progress.max = change.new
    
    def load_examples(self, b):
        """Load a random set of examples from source data"""
        n = self.num_examples_input.value
        if n <= 0:
            self.status_text.value = "<p style='color:red;'>Please enter a positive number of examples.</p>"
            return
        
        if not self.source_data:
            self.status_text.value = "<p style='color:red;'>No source data available. Check the source file.</p>"
            return
            
        # Filter out already processed examples
        unprocessed_data = []
        for item in self.source_data:
            conversation_id = self.get_conversation_id(item)
            if conversation_id not in self.processed_ids:
                unprocessed_data.append(item)
        
        if not unprocessed_data:
            self.status_text.value = "<p style='color:orange;'>All source data has already been processed!</p>"
            return
            
        # Shuffle and select examples
        random.shuffle(unprocessed_data)  # Randomize the entire dataset
        self.examples_to_process = unprocessed_data[:min(n, len(unprocessed_data))]
        self.total_to_process = len(self.examples_to_process)
        self.current_index = 0
        self.progress.max = self.total_to_process
        self.progress.value = 0
        
        # Load first example
        self.load_current_example()
        
        self.status_text.value = f"<p style='color:green;'>Loaded {self.total_to_process} examples. Processing example 1/{self.total_to_process}</p>"
    
    def get_conversation_id(self, example):
        """Extract conversation ID from an example"""
        # First check for conversation_id field
        if 'conversation_id' in example and example['conversation_id']:
            return str(example['conversation_id'])
            
        # Try other possible ID fields
        id_column_names = ['id', '_id', 'index', 'prompt_id']
        for col in id_column_names:
            if col in example and example[col]:
                return str(example[col])
                
        # If no ID found, create a hash from the prompt text
        for col in ['prompt', 'text', 'content', 'query', 'question', 'input']:
            if col in example and example[col] and isinstance(example[col], str):
                return str(hash(example[col]))
                
        # Last resort - return a random string
        return str(random.randint(10000, 99999))
    
    def load_current_example(self):
        """Load the current example into the UI"""
        if self.current_index < len(self.examples_to_process):
            example = self.examples_to_process[self.current_index]
            
            # Try various column names that might contain the prompt
            prompt_column_names = ['prompt', 'text', 'content', 'query', 'question', 'input']
            
            # Find prompt text
            prompt_text = None
            for col in prompt_column_names:
                if col in example and example[col] and isinstance(example[col], str):
                    prompt_text = example[col]
                    break
            
            # If no prompt found, use the first string value
            if not prompt_text:
                for key, value in example.items():
                    if isinstance(value, str) and len(value) > 10:
                        prompt_text = value
                        break
            
            # Get the conversation ID
            conversation_id = self.get_conversation_id(example)
                
            # Set the values in the UI
            self.prompt_text.value = prompt_text if prompt_text else ""
            self.conversation_id_text.value = conversation_id
            self.context_text.value = ""
            self.instruction_preview.value = ""
            
            # Reset current prompt
            self.current_prompt = None
            self.preview.value = "<p><b>Preview:</b></p><p>Please paste the context portion or select 'No Context'</p>"
    
    def extract_context(self, b):
        """Extract context from the prompt based on pasted text"""
        prompt = self.prompt_text.value
        context = self.context_text.value.strip()
        
        if not prompt:
            self.status_text.value = "<p style='color:red;'>No prompt loaded yet.</p>"
            return
            
        if not context:
            self.status_text.value = "<p style='color:red;'>Please paste a context portion first.</p>"
            return
        
        if context not in prompt:
            self.status_text.value = "<p style='color:red;'>Context text not found in the prompt. Please copy exact text.</p>"
            return
            
        # Find where the context appears in the prompt
        start_pos = prompt.find(context)
        end_pos = start_pos + len(context)
        
        # Split the prompt into context and instruction
        instruction = prompt[:start_pos] + prompt[end_pos:]
        instruction = instruction.strip()
        
        # Update the instruction preview
        self.instruction_preview.value = instruction
        
        self.current_prompt = {
            "original_prompt": prompt,
            "context": context,
            "instruction": instruction,
            "has_context": True,
            "conversation_id": self.conversation_id_text.value
        }
        
        # Update preview
        self.update_preview()
        self.status_text.value = "<p style='color:green;'>Context extracted successfully!</p>"
    
    def mark_no_context(self, b):
        """Mark this prompt as having no context"""
        prompt = self.prompt_text.value
        
        if prompt:
            self.context_text.value = ""
            self.instruction_preview.value = prompt
            
            self.current_prompt = {
                "original_prompt": prompt,
                "context": "",
                "instruction": prompt,
                "has_context": False,
                "conversation_id": self.conversation_id_text.value
            }
            
            # Update preview
            self.update_preview()
            self.status_text.value = "<p style='color:blue;'>Prompt marked as having no context.</p>"
        else:
            self.status_text.value = "<p style='color:red;'>Please enter a prompt first.</p>"
    
    def skip_prompt(self, b):
        """Skip the current prompt and load the next one"""
        self.current_index += 1
        self.progress.value = self.current_index
        
        if self.current_index < self.total_to_process and self.current_index < len(self.examples_to_process):
            # Load next example
            self.load_current_example()
            self.status_text.value = f"<p style='color:orange;'>Skipped! Processing example {self.current_index+1}/{self.total_to_process}</p>"
        else:
            # All done
            self.prompt_text.value = ""
            self.context_text.value = ""
            self.instruction_preview.value = ""
            self.conversation_id_text.value = ""
            self.current_prompt = None
            self.preview.value = "<p><b>Preview:</b></p><p>All examples processed!</p>"
            self.status_text.value = f"<p style='color:green;'>All done! Processed {len(self.prompts_data)} prompts. Click 'Export to CSV' to save.</p>"
    
    def next_prompt(self, b):
        """Save current prompt and move to next"""
        if self.current_prompt:
            # Add to processed IDs to avoid duplicates
            self.processed_ids.add(self.current_prompt["conversation_id"])
            
            self.prompts_data.append(self.current_prompt)
            self.current_index += 1
            self.progress.value = self.current_index
            
            if self.current_index < self.total_to_process and self.current_index < len(self.examples_to_process):
                # Load next example
                self.load_current_example()
                self.status_text.value = f"<p style='color:green;'>Saved! Processing example {self.current_index+1}/{self.total_to_process}</p>"
            else:
                # All done
                self.prompt_text.value = ""
                self.context_text.value = ""
                self.instruction_preview.value = ""
                self.conversation_id_text.value = ""
                self.current_prompt = None
                self.preview.value = "<p><b>Preview:</b></p><p>All examples processed!</p>"
                self.status_text.value = f"<p style='color:green;'>All done! Processed {len(self.prompts_data)} prompts. Click 'Export to CSV' to save.</p>"
        else:
            self.status_text.value = "<p style='color:red;'>Please extract context or select 'No Context' first.</p>"
    
    def export_to_csv(self, b):
        """Export all processed prompts to CSV"""
        if self.prompts_data:
            # Create directory if it doesn't exist
            output_dir = os.path.join("..", "data", "prompt_training_2.0")
            os.makedirs(output_dir, exist_ok=True)
            
            # Get filename from input or use timestamp
            filename = self.filename_input.value.strip()
            if not filename:
                # Create timestamp
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"{timestamp}_separated_prompts.csv"
            elif not filename.endswith('.csv'):
                filename += '.csv'
                
            filepath = os.path.join(output_dir, filename)
            
            # Create dataframe and save
            df = pd.DataFrame(self.prompts_data)
            df.to_csv(filepath, index=False)
            
            self.status_text.value = f"<p style='color:green;'>Exported {len(self.prompts_data)} prompts to {filepath}</p>"
            
            # Show a preview of the dataframe
            display(df.head())
        else:
            self.status_text.value = "<p style='color:red;'>No prompts to export!</p>"
    
    def update_preview(self):
        """Update the preview of the current prompt separation"""
        if self.current_prompt:
            prompt = self.current_prompt
            
            if prompt["has_context"]:
                preview_html = f"""
                <p><b>Preview:</b></p>
                <p><b>Conversation ID:</b> {prompt["conversation_id"]}</p>
                <p><b>Original Prompt:</b> {prompt["original_prompt"]}</p>
                <div style='margin-top:10px; padding:10px; background-color:#e6f7ff; border-left:4px solid #1890ff;'>
                    <p><b>Context:</b></p>
                    <p>{prompt["context"]}</p>
                </div>
                <div style='margin-top:10px; padding:10px; background-color:#f6ffed; border-left:4px solid #52c41a;'>
                    <p><b>Instruction:</b></p>
                    <p>{prompt["instruction"]}</p>
                </div>
                """
            else:
                preview_html = f"""
                <p><b>Preview:</b></p>
                <p><b>Conversation ID:</b> {prompt["conversation_id"]}</p>
                <p><b>Original Prompt:</b> {prompt["original_prompt"]}</p>
                <div style='margin-top:10px; padding:10px; background-color:#f6ffed; border-left:4px solid #52c41a;'>
                    <p><b>Instruction (No Context):</b></p>
                    <p>{prompt["instruction"]}</p>
                </div>
                """
            
            self.preview.value = preview_html
    
    def display(self):
        """Display the app"""
        display(self.app)


# Create and display the app
separator = PromptSeparator()
separator.display()

Loaded 34358 prompts from ..\data\prompt_training_2.0\candidate_train_data.csv


,original_prompt,context,instruction,has_context,conversation_id
0,I want you to generate a prompt for me identif...,NaN,I want you to generate a prompt for me identif...,False,23321
1,I am the CEO of a C2C app startup. Tell me wha...,NaN,I am the CEO of a C2C app startup. Tell me wha...,False,25547
2,\n \n \n \n \n 1 / 1\n \...,The term Product is mostly used as a need-sati...,1 / 1\n \n \n \n simplify:,True,18484
3,Provide complete and working implementation f...,NaN,Provide complete and working implementation f...,False,16597
4,Using industry standard procurement best pract...,NaN,Using industry standard procurement best pract...,False,184


,original_prompt,context,instruction,has_context,conversation_id
0,I want you to generate a prompt for me identif...,NaN,I want you to generate a prompt for me identif...,False,23321
1,I am the CEO of a C2C app startup. Tell me wha...,NaN,I am the CEO of a C2C app startup. Tell me wha...,False,25547
2,\n \n \n \n \n 1 / 1\n \...,The term Product is mostly used as a need-sati...,1 / 1\n \n \n \n simplify:,True,18484
3,Provide complete and working implementation f...,NaN,Provide complete and working implementation f...,False,16597
4,Using industry standard procurement best pract...,NaN,Using industry standard procurement best pract...,False,184


,original_prompt,context,instruction,has_context,conversation_id
0,I want you to generate a prompt for me identif...,NaN,I want you to generate a prompt for me identif...,False,23321
1,I am the CEO of a C2C app startup. Tell me wha...,NaN,I am the CEO of a C2C app startup. Tell me wha...,False,25547
2,\n \n \n \n \n 1 / 1\n \...,The term Product is mostly used as a need-sati...,1 / 1\n \n \n \n simplify:,True,18484
3,Provide complete and working implementation f...,NaN,Provide complete and working implementation f...,False,16597
4,Using industry standard procurement best pract...,NaN,Using industry standard procurement best pract...,False,184


,original_prompt,context,instruction,has_context,conversation_id
0,I want you to generate a prompt for me identif...,NaN,I want you to generate a prompt for me identif...,False,23321
1,I am the CEO of a C2C app startup. Tell me wha...,NaN,I am the CEO of a C2C app startup. Tell me wha...,False,25547
2,\n \n \n \n \n 1 / 1\n \...,The term Product is mostly used as a need-sati...,1 / 1\n \n \n \n simplify:,True,18484
3,Provide complete and working implementation f...,NaN,Provide complete and working implementation f...,False,16597
4,Using industry standard procurement best pract...,NaN,Using industry standard procurement best pract...,False,184


...

,original_prompt,context,instruction,has_context,conversation_id
0,I want you to generate a prompt for me identif...,NaN,I want you to generate a prompt for me identif...,False,23321
1,I am the CEO of a C2C app startup. Tell me wha...,NaN,I am the CEO of a C2C app startup. Tell me wha...,False,25547
2,\n \n \n \n \n 1 / 1\n \...,The term Product is mostly used as a need-sati...,1 / 1\n \n \n \n simplify:,True,18484
3,Provide complete and working implementation f...,NaN,Provide complete and working implementation f...,False,16597
4,Using industry standard procurement best pract...,NaN,Using industry standard procurement best pract...,False,184


,original_prompt,context,instruction,has_context,conversation_id
0,I want you to generate a prompt for me identif...,NaN,I want you to generate a prompt for me identif...,False,23321
1,I am the CEO of a C2C app startup. Tell me wha...,NaN,I am the CEO of a C2C app startup. Tell me wha...,False,25547
2,\n \n \n \n \n 1 / 1\n \...,The term Product is mostly used as a need-sati...,1 / 1\n \n \n \n simplify:,True,18484
3,Provide complete and working implementation f...,NaN,Provide complete and working implementation f...,False,16597
4,Using industry standard procurement best pract...,NaN,Using industry standard procurement best pract...,False,184


,original_prompt,context,instruction,has_context,conversation_id
0,I want you to generate a prompt for me identif...,NaN,I want you to generate a prompt for me identif...,False,23321
1,I am the CEO of a C2C app startup. Tell me wha...,NaN,I am the CEO of a C2C app startup. Tell me wha...,False,25547
2,\n \n \n \n \n 1 / 1\n \...,The term Product is mostly used as a need-sati...,1 / 1\n \n \n \n simplify:,True,18484
3,Provide complete and working implementation f...,NaN,Provide complete and working implementation f...,False,16597
4,Using industry standard procurement best pract...,NaN,Using industry standard procurement best pract...,False,184


,original_prompt,context,instruction,has_context,conversation_id
0,I want you to generate a prompt for me identif...,NaN,I want you to generate a prompt for me identif...,False,23321
1,I am the CEO of a C2C app startup. Tell me wha...,NaN,I am the CEO of a C2C app startup. Tell me wha...,False,25547
2,\n \n \n \n \n 1 / 1\n \...,The term Product is mostly used as a need-sati...,1 / 1\n \n \n \n simplify:,True,18484
3,Provide complete and working implementation f...,NaN,Provide complete and working implementation f...,False,16597
4,Using industry standard procurement best pract...,NaN,Using industry standard procurement best pract...,False,184


,original_prompt,context,instruction,has_context,conversation_id
0,I want you to generate a prompt for me identif...,NaN,I want you to generate a prompt for me identif...,False,23321
1,I am the CEO of a C2C app startup. Tell me wha...,NaN,I am the CEO of a C2C app startup. Tell me wha...,False,25547
2,\n \n \n \n \n 1 / 1\n \...,The term Product is mostly used as a need-sati...,1 / 1\n \n \n \n simplify:,True,18484
3,Provide complete and working implementation f...,NaN,Provide complete and working implementation f...,False,16597
4,Using industry standard procurement best pract...,NaN,Using industry standard procurement best pract...,False,184
